# Exercises

## Setup

In [ ]:
import arviz as az
import bambi as bmb
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

from bambi.plots import plot_cap
from matplotlib.lines import Line2D

In [ ]:
%matplotlib inline
plt.style.use("intuitivebayes.mplstyle")

mpl.rcParams["figure.dpi"] = 120
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["axes.spines.left"] = False

## Exercise 1: Art-attack

In the section "The world's simplest model" from "Lesson 2: Regression Refresher" we created the following visualization.

<center>
    <img src="imgs/intercept_only_model_curve.png" style="width:950px;"/>
</center>

This allowed us to see the intercept-only model fits a flat line that is not affected by the values of the predictor. 

You're asked to reproduce this visualization, using the Bambi model we created in this lesson. Fortunately, you don't need to write a lot of code from scratch. Use the snippet from Lesson 2 and find the places where you need to change parameter names. 

In [ ]:
# We've loaded the data for you
fish_data = pd.read_csv("data/fish-market.csv")
fish_data[fish_data["Weight"] > 0].reset_index(drop=True)
fish_data.head()

In [ ]:
# Write Bambi model here


### Reference Python Code

```python
fig, ax = plt.subplots(figsize=FIGSIZE)
ax.scatter(x=data["Length1"], y=data["Weight"], alpha=0.6)
ax.set(xlabel="Length (centimeters)", ylabel="Weight (grams)", title="Fish length vs weight");

for value in idata.posterior["β0"].to_numpy().flatten()[::10]:
    ax.axhline(value, color="C1", alpha=0.2)

β0_mean = idata.posterior["β0"].to_numpy().mean()
ax.axhline(β0_mean, color="C4")

handles = [
    Line2D([], [], label="Observations", lw=0, marker="o", color="C0", alpha=0.6),
    Line2D([], [], label="Posterior draws", lw=2, color="C1"),
    Line2D([], [], label="Posterior mean", lw=2, color="C4"),
]
ax.legend(handles=handles, loc="upper left")
```

## Exercise 2: Mastering `plot_cap()`

In this exercise you have to work with the fish data and the model we created in the "Transformations in Bambi" section. More concretely, you need to build the following model

```python
model = bmb.Model("log(Weight) ~ 1 + log(Length1)", data)
```
Notice you need to import the data to solve this exercise. Then create and fit the model mentioned above, and use `plot_cap()` with `Length1` as the variable for the horizontal axis.

```python
plot_cap(model, idata, "Length1")
```

Answer the following questions

* What do you see?
    * Is it a straight line?
    * Why?
* What is the scale of the variables in the horizontal and the vertical axis? Consider they're used with an inline transformation in the model.
    * Why?
* How can you put both variables in the transformed scale?
* And how you put everything on the untransformed scale?

In [1]:
# Build Bambi model here

In [2]:
# Create a cap plot

## Exercise 3: Advanced operators

In the section "Transformations in Bambi" we showed that some custom operations needed to be written using a particular syntax. In this exercise you're going to experiment a little with with these operations to see how they work in practice. The goal here is not to arrive to a "correct" answer, but to see what are the different results when we try different formulas and trying to understand what's going on. Use the following simulated data:

In [ ]:
rng = np.random.default_rng(1211)
size = 100
df = pd.DataFrame(
    {
        "response": rng.normal(loc=5, scale=0.5, size=size),
        "x": rng.normal(loc=10, scale=2, size=size),
        "y": rng.normal(size=size),
    }
)


### Part 1
Use the following formulas and explain what's happening

* `"response ~ x ** 100"`
* `"response ~ x / y"`
* `"response ~ (x + y) ** 2"`

It's fine if you can only describe what you see. At this point you're not expected to be an expert in formula notation. We'll cover more advanced stuff later in the course.

In [ ]:
# Write code here

### Part 2
Try this
* `"response ~ x / 100`"

You'll see an exception, this is expected. We want you to get familiar with what it looks like when Bambi formula syntax fails, and give you a preview to later lessons in the course!

In [ ]:
# Write code here

### Part 3
What is the difference between `"response ~ x + y"` and `"response ~ I(x + y)"`?

In [3]:
# Write code here

## Exercise 4: Time to experiment

In the "Parameter identifiability" section we simulated some data to grasp what non-identifiability means using a controlled scenario. The code we used is the following:

In [ ]:
rng = np.random.default_rng(1234)
b0, b1 = 0.5, 2.5
x = np.linspace(0, 3, num=50)
noise = rng.normal(scale=0.5, size=50)
y = b0 + b1 * x + noise

And then we created the following PyMC model

In [ ]:
with pm.Model() as weird_model:
    b0 = pm.Normal("b0", sigma=0.5)
    b1 = pm.Normal("b1")
    b2 = pm.Normal("b2")
    mu = b0 + b1 * x + b2 * x
    sigma = pm.HalfNormal("sigma" ,sigma=0.5)
    pm.Normal("y", mu=mu, sigma=sigma, observed=y)

### Part 1

The goal of this exercise is to have you run several experiments to understand how parameter non-identifiablity affects sampling times and the quality of the draws obtained. You are asked to

* Repeat this experiment with different sample sizes.
    * Use $n=500$ 
    * Use $n=2000$
* Explore the posterior of the slopes
* Do you get any warnings?
* Explore the correlation between $b_1$ and $b_2$. 
    * Does it improve with more data?

In [ ]:
# Write code here

### Part 2

Now, use the correct model, with a single slope

```python
with pm.Model() as good_model:
    b0 = pm.Normal("b0", sigma=0.5)
    b1 = pm.Normal("b1")
    mu = b0 + b1 * x
    sigma = pm.HalfNormal("sigma" ,sigma=0.5)
    pm.Normal("y", mu=mu, sigma=sigma, observed=y)
```

* Fit it using $n=500$ and $n=2000$, as done previously
* Compare the sampling speed with the previous model
    * What can you conclude about the effects of parameter correlations in sampling speed?
* Can you recover the true slope?
* Do you get any warnings?
* Also explore the correlation between model parameters. 
    * What can you conclude?

Finally, you're asked to reproduce the flawed model in Bambi. To do so, you need to use the formula `"y ~ x + x"`. 

* What happens?
* Why do you think it works that way?

In [ ]:
# Write code here

## Exercise 5: What if...

In the section "An end to end trip with Bambi" we computed the $R^2$ coefficient of the following model

```python
model = bmb.Model("log(Weight) ~ 0 + Species + log(Length1):Species", data)
```

We got an $R^2$ equal to 0.984, which is very high. 

In this exercise you're asked to use the other numerical predictors in the dataset in place of `Length1`, compute the $R^2$ of those models, and compare to what we got with `Length1`.

**Notes**

You have to create two extra models. One using `Height` and other using `Width` as predictors. You can keep the same strcuture of varying intercepts and varying slopes. 

**Optional**

Combine two or more of the numerical predictors, compute the $R^2$ coefficient, and compare with the previous models. Does adding more numerical predictors improve $R^2$?

In [ ]:
# Write code here

## Exercise 6: Interaction effects

In this exercise we're going to learn more about interaction effects. In the section "The full model" we covered that the different `log(Length)` slopes per `Species` is an interaction between `log(Length)` and `Species`. More generally, this is an interaction between a numeric and a categorical variable. The result was as many slopes as species in our dataset.

Now we're going to exercise another type of interaction, between two numerics, which is the first case in the following diagram.

<center>
  <img src="imgs/interaction_symbol.png" style="width:950px"; />
</center>

We expect this to give us a single new slope, which is multiplied by the product of the two numerical covariates.

### The data

In [ ]:
mtcars_data = pd.read_csv("data/mtcars.csv")
mtcars_data.head()

The data was extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973--74 models). The following is a description of the variables:

* `mpg`: Miles/(US) gallon
* `cyl`: Number of cylinders
* `disp`: Displacement (cu.in.)
* `hp`: Gross horsepower
* `drat`: Rear axle ratio
* `wt`: Weight (1000 lbs)
* `qsec`: 1/4 mile time
* `vs`: Engine (0 = V-shaped, 1 = straight)
* `am`: Transmission (0 = automatic, 1 = manual)
* `gear`: Number of forward gears

## The models

We're going to us the Gross horsepower (`hp`) and the Weight (`wt`) to predict the fuel consumption in Miles per US gallon (`mpg`).

**Main effects model**

$$
\text{mpg} = \beta_0 + \beta_1 \text{hp} + \beta_2 \text{wt} + \varepsilon
$$

**Interaction effects model**

$$
\text{mpg} = \beta_0 + \beta_1 \text{hp} + \beta_2 \text{wt} + \beta_3 \text{hp} \cdot \text{wt}  + \varepsilon
$$

### The questions

* Build and fit the main effects model
* Build and fit the model with the interaction effects
* Explore the posterior of both models
    * What can you conclude about the coefficient of the interaction effect?
* Use `plot_cap()` to compare the fitted curves
    * Map `hp` to the horizontal axis and `wt` to the color or the panel.
    * Can you describe how the interaction affects the fitted curves?

In [ ]:
# Write code here

## Exercise 7: A model with many predictors 

This is the continuation of Exercise 4 from the previous lesson, where we worked with the construction problem. Now we're going to use all the predictors, not just cement and water, to predict the concrete strength. 

The goal of this exercise is to learn how scaling of the predictors may help our sampler to work faster. Also, it challenges you to come up with an alternative that will prevent our model from giving negative predictions, a problem we already detected in Exercise 4 from the previous lesson.

If you want to refresh your knowledge about the `scale()` transform, revisit the section "Transformations in Bambi".

### The data

The team has already tested more than a thousand samples ([source](https://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength)) and the following variables were measured and recorded

* **cement** - Portland cement in kg/m3
* **slag** - Blast furnace slag in kg/m3
* **fly_ash** - Fly ash in kg/m3
* **water** - Water in liters/m3
* **superplasticizer** - Superplasticizer additive in kg/m3
* **coarse_aggregate** - Coarse aggregate (gravel) in kg/m3
* **fine_aggregate** - Fine aggregate (sand) in kg/m3
* **age** - Age of the sample in days
* **strength** - Concrete compressive strength in megapascals (MPa)

In [ ]:
concrete_data = pd.read_csv("data/concrete.csv")
concrete_data.head()

In [ ]:
concrete_data_new = pd.DataFrame(
    {
        "cement": [520, 300, 400],
        "slag": [100, 50, 70],
        "fly_ash": [120, 40, 90],
        "water": [228, 160, 200],
        "superplasticizer": [22, 16, 20],
        "coarse_aggregate": [1000, 800, 900],
        "fine_aggregate": [825, 650, 775],
        "age": [48, 128, 80],
    }
)

### The models

Since we're using all the predictors, the formulas are quite big. We could build them using Python, but for the sake of clarity we prefer to just write them down.

**Unscaled model**

```python
formula = "strength ~ cement + slag + fly_ash + water + superplasticizer + coarse_aggregate + fine_aggregate + age"
```

**Scaled model**

```python
formula = "strength ~ scale(cement) + scale(slag) + scale(fly_ash) + scale(water) + scale(superplasticizer) + scale(coarse_aggregate) + scale(fine_aggregate) + scale(age)"
```

### The questions

* Build and fit the unscaled model
* Build and fit the scaled model
* Compare sampling times
* Predict the weight of the three samples in the data frame `concrete_data_new` using both models
    * Do predictions differ between models?
    * Why?
* Plot the posterior predictive distribution
    * If the model predicts negative strenghts, propose a model that fixes this problem.
    * Fit the new model and explore the posterior predictive distribution again.
    * What can you conclude?

In [ ]:
# Write code here

### Citations

The data for this exercise originally comes from

* I-Cheng Yeh, "Modeling of strength of high performance concrete using artificial neural networks," Cement and Concrete Research, Vol. 28, No. 12, pp. 1797-1808 (1998).